# Run Exponential Smoothing Model
Steps:
1. Open dataset file
2. Run ES model (including save to file)
3. Evaluation

# 1. Open Dataset File

## Import libraries

In [1]:
# Import file and data libraries
import math
import pickle
import numpy as np
import pandas as pd 
import collections

# Import exponential smoothing model
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   # For single ES
from statsmodels.tsa.holtwinters import ExponentialSmoothing # For double & triple ES

# Import Matthews Correlation Coefficient module (MCC)
from sklearn.metrics import matthews_corrcoef

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Read dataset files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read X & Y datasets from Pickle file, along with the metadata
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset.pickle", "rb") as f:
    X = pickle.load(f)
    Y = pickle.load(f)  # Contains Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
    Metadata = pickle.load(f) # Contains the Company ID, ticker, sector and Dataset ID
f.close()

# 2. Run Exponential Smoothing model

## Create functions

In [4]:
# Function to calculate 5% band i.e. 
#     0 to 5% = U1 (including 0), 5 to 10% = U2, 10 to 15% = U3, 15 to 20% = U4, >= U5.
#     <0 to -5% = D1, -5 to -10% = D2, -10 to -15% = D3, -15 to -20 = D4, <-20% = D5.
def calc_band(input_pct):
    if input_pct > 20: 
        band = 5
        prefix = 'U'
    elif input_pct < -20: 
        band = 5
        prefix = "D"
    elif input_pct >= 0:
        band = math.floor(input_pct / 5) + 1 # Floor(1/5) is 0 so to make it Band 1 we need to add 1
        prefix = "U"
    elif input_pct < 0:
        band = math.floor(input_pct / 5) # Floor(-1/5) is -1 so no need to add 1
        prefix = "D"
    else: # NaN for example
        band = 0
        prefix = "E" # Error band
    return prefix + str(abs(band))

In [5]:
# Create a function to:
#   - Forecast price band using ES model
#   - Write the forecasted and actual band to output file

# Parameters: 
#   - Number of Samples (int): number of loop, i.e. 100 (default) or 1000
#   - Model (string): "Simple ES", "Double ES" (default) or "Triple ES"
#   - Mode (string): "A" for additive (default) and "M" for Multiplicative 
#                    (only applicable for DES and TES, for SES model this parameter is ignored)

# Returns:
#   - Result DF: Result data frame containing the forecasted & actual bands, and status

def Forecast_Using_ES_Model (Number_of_Samples = 100, Model = "Double ES", Mode = "A"):
    Y_Band_List   = [] # List of forecast band
    F_Band_List   = [] # List of actual band
    F_Status_List = [] # List of forecast status
    Output_File   = "" # List of output rows to be saved to file

    # Forecast the price band for the first 1000 datasets (the remaining 2989 datasets are for ML training)
    for ds in range(Number_of_Samples):
    
        # Set the date as the index 
        X_ES = X[ds][["Date","HL_Avg"]].set_index("Date")
        
        # Set the period to 1 year
        Period = int(len(X_ES)/5) 
        
        # Set the text for additive or multiplicative based on the Mode parameter
        if Mode == "A": Mode_Text = "add"
        elif Mode == "M": Mode_Text = "mul"
        else: Mode_Text = "add"  # for any other value use additive

        # Create an ES model
        if Model == "Simple ES":
            fitted_model = SimpleExpSmoothing(X_ES).fit()
        elif Model == "Double ES":
            fitted_model = ExponentialSmoothing(X_ES, trend = Mode_Text).fit()
        elif Model == "Triple ES":
            fitted_model = ExponentialSmoothing(X_ES, trend = Mode_Text, seasonal = Mode_Text, seasonal_periods=Period).fit()
        else: # for any other value it is use Double ES
            fitted_model = ExponentialSmoothing(X_ES, trend = Mode_Text).fit()
    
        # Y variable consists of: Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days
        Forecast_From = len(X_ES) + int(Y[ds][4]) # Y[4] is Y_Gap (number of days from X Last Price). Tipically 64.
        Forecast_To   = Forecast_From + int(Y[ds][5]) # Y[5] is Y_Days (number of days in a week for Y). Tipically 5.

        F = fitted_model.forecast(Forecast_To)

        # Calculate forecast price band
        F_avg = F[Forecast_From:Forecast_To].mean() # Average the 1 week prices (tipically 65 to 69)
        X_last = float(Y[ds][1]) # Y(1) is the last price in X 5 year range
        F_pct = (F_avg - X_last) / X_last * 100
        F_band = calc_band(F_pct) # The forecast band

        # Calculate Y band
        Y_band = Y[ds][3] #Y[3] is Y_Band (the actual price band) i.e. D1, U1, D2, U2, etc.

        # Compare the forecast to actual
        if F_band == Y_band: F_status = 1
        else: F_status = 0
    
        # Store the actual, forecast and comparison status
        Y_Band_List.append(Y_band)
        F_Band_List.append(F_band)
        F_Status_List.append(F_status)

        # Collect the output rows into 1 variable to be written to the output file
        Output_Row = str(ds) + "," + F_band + "," + Y_band + "," + str(F_status) + "\n"
        Output_File = Output_File + Output_Row

    # Save the forecast and actual bands into a dataframe for evaluation
    Result_DF = pd.DataFrame({"Y Band"   : Y_Band_List, 
                              "F Band"   : F_Band_List, 
                              "F Status" : F_Status_List})
    return Result_DF

## Forecast using Simple ES model

In [6]:
# Setting the number of datasets to run (the same number for all 5 ES models)
Datasets_to_Run = 1000
# 100  takes about  2 minutes (1s for simple, 3s & 6s for double, 40s & 1m for triple)
# 1000 takes about 32 minutes (7s for simple, 34s & 46s for double, 5m & 25m for triple)

In [7]:
# Forecast using a Simple Exponential Smoothing model (8 seconds)
Result_Simple_ES_DF   = Forecast_Using_ES_Model(Datasets_to_Run, "Simple ES")

## Forecast using Double ES model

In [8]:
# Forecast using a Double Exponential Smoothing Additive model (38 seconds)
Result_Double_ES_A_DF = Forecast_Using_ES_Model(Datasets_to_Run, "Double ES", "A")

In [9]:
# Forecast using a Double Exponential Smoothing Multiplicative model (50 seconds)
Result_Double_ES_M_DF = Forecast_Using_ES_Model(Datasets_to_Run, "Double ES", "M")

## Forecast using Triple ES model

In [10]:
# Forecast using a Triple Exponential Smoothing Additive model (6 minutes)
Result_Triple_ES_A_DF = Forecast_Using_ES_Model(Datasets_to_Run, "Triple ES", "A")

In [11]:
# Forecast using a Double Exponential Smoothing Multiplicative model
Result_Triple_ES_M_DF = Forecast_Using_ES_Model(Datasets_to_Run, "Triple ES", "M")

# Save to CSV file

In [13]:
Result_Simple_ES_DF["Y Band"][5]

'D3'

In [16]:
# Collect the output rows into 1 variable to be written to the output file
Output_File = ""
for DS in range(Datasets_to_Run):
    Output_Row = str(DS) + "," + Result_Simple_ES_DF["Y Band"][DS] + "," + \
                 Result_Simple_ES_DF["F Band"][DS]   + "," + str(Result_Simple_ES_DF["F Status"][DS]) + "," + \
                 Result_Double_ES_A_DF["F Band"][DS] + "," + str(Result_Double_ES_A_DF["F Status"][DS]) + "," + \
                 Result_Double_ES_M_DF["F Band"][DS] + "," + str(Result_Double_ES_M_DF["F Status"][DS]) + "," + \
                 Result_Triple_ES_A_DF["F Band"][DS] + "," + str(Result_Triple_ES_A_DF["F Status"][DS]) + "," + \
                 Result_Triple_ES_M_DF["F Band"][DS] + "," + str(Result_Triple_ES_M_DF["F Status"][DS]) + "\n"
    Output_File = Output_File + Output_Row

In [17]:
# Save output into file
File_Path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Output/"
f = open(File_Path + "ES.csv", "w")
f.write(Output_File)
f.close()


# 3. Evaluation

## Calculate %Correct

In [ ]:
# The final result (how many got right out)
print("Simple ES :", sum(Result_Simple_ES_DF["F Status"])) # 126
print("Double ES Additive :", sum(Result_Double_ES_A_DF["F Status"])) # 164
print("Double ES Multiplicative:", sum(Result_Double_ES_M_DF["F Status"])) # 161
print("Triple ES Additive :", sum(Result_Triple_ES_A_DF["F Status"])) # 156
print("Triple ES Multiplicative:", sum(Result_Triple_ES_M_DF["F Status"])) # 152

Simple ES : 126
Double ES Additive : 164
Double ES Multiplicative: 161
Triple ES Additive : 156
Triple ES Multiplicative: 152


## Calculate Matthew Corr Coef

In [ ]:
# Calculate the MCC for Simple Exponential Smoothing
MCC_SES = matthews_corrcoef(Result_Simple_ES_DF["Y Band"], Result_Simple_ES_DF["F Band"])
print("MCC for Simple ES : ", MCC_SES) #-0.0023346840840233717

MCC for Simple ES :  -0.0023346840840233717


In [ ]:
# Calculate the MCC for Double Exponential Smoothing
MCC_DES_A = matthews_corrcoef(Result_Double_ES_A_DF["Y Band"], Result_Double_ES_A_DF["F Band"])
print("MCC for Double ES A: ", MCC_DES_A) #0.00772237371229578

MCC_DES_M = matthews_corrcoef(Result_Double_ES_M_DF["Y Band"], Result_Double_ES_M_DF["F Band"])
print("MCC for Double ES M: ", MCC_DES_M) #0.02983851588491017

MCC for Double ES :  0.00772237371229578
MCC for Double ES :  0.02983851588491017


In [ ]:
# Calculate MCC for Triple Exponential Smoothing
MCC_TES_A = matthews_corrcoef(Result_Triple_ES_A_DF["Y Band"], Result_Triple_ES_A_DF["F Band"])
print("MCC for Triple ES A: ", MCC_TES_A) #0.014371324088720664

MCC_TES_M = matthews_corrcoef(Result_Triple_ES_M_DF["Y Band"], Result_Triple_ES_M_DF["F Band"])
print("MCC for Triple ES M : ", MCC_TES_M) #0.006429931071526978

MCC for Double ES :  0.014371324088720664
MCC for Double ES :  0.006429931071526978
